In [1]:
import pandas as pd
import glob
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
#df_g = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/t20_g.csv')
df_g = pd.read_csv('/Users/roumyadas/Desktop/IPL_Simulation/Season_03/Stats/df_combined_SPL.csv')

## ground stats only from SPL

df_g.columns

In [5]:
df_g['phase'] = np.where(df_g['ball']<37, 'pp', 
                        np.where(df_g['ball']>=90, 'death',
                            'middle'))

In [6]:
df_g['phase'].value_counts()

middle    15620
pp        10656
death      8720
Name: phase, dtype: int64

In [7]:
df_g[df_g.phase=='pp'].ball.describe()

count    10656.000000
mean        18.500000
std         10.388782
min          1.000000
25%          9.750000
50%         18.500000
75%         27.250000
max         36.000000
Name: ball, dtype: float64

In [8]:
df_g.venue.value_counts()

CSK     3971
PBKS    3936
SRH     3845
DC      3376
LSG     3376
MI      3353
KKR     3328
GT      3298
RCB     3292
RR      3221
Name: venue, dtype: int64

In [9]:
df_g.columns

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'wicket_type', 'player_dismissed', 'legal_balls_bowled', 'wicket_event',
       'fielder', 'legality', 'bowler_wicket', 'islegal', 'isDotforBatter',
       'isOne', 'isTwo', 'isThree', 'isStrikeRotation', 'isFour', 'isSix',
       'isBowlerWicket', 'is_striker_Out', 'is_nonstriker_Out', 'isWicket',
       'total_runs', 'runs_scored', 'wickets_down', 'run_rate', 'last_fow',
       'partnership_runs', 'isno', 'is_faced_by_batter', 'last_fow_run_rate',
       'fours', 'sixes', 'striker_batting_position',
       'non_striker_batting_position', 'runs_conceeded', 'isDotforbowler',
       'legal_balls_remaining', 'target', 'runs_remaining', 'reqd_run_rate',
       'pitch', 'toss', 'decision', 'phase'],
      dtype='object')

In [10]:
ground_stats = df_g.groupby(['venue','phase','innings']).agg(
    num_innings = ('match_id','nunique'),
    runs = ('total_runs','sum'),
    balls = ('islegal' ,'sum'),
    wickets = ('isWicket','sum'),
    fours = ('isFour', 'sum'),
    sixes = ('isSix','sum'),
    dots = ('isDotforBatter','sum'),
    extras = ('extras','sum'),
    ones = ('isOne','sum'),
    twos = ('isTwo','sum'),
    threes = ('isThree','sum'),
    wides = ('wides','sum'),
    noballs = ('noballs','sum'),
    byes = ('byes','sum'),
    legbyes = ('legbyes','sum')
    
    
    
    
).reset_index()

ground_stats['run_rate'] = 6*ground_stats['runs']/ground_stats['balls']
ground_stats['strike_rate'] = 100*ground_stats['runs']/ground_stats['balls']
ground_stats['bowl_strike_rate'] = ground_stats['balls']/ground_stats['wickets']
ground_stats['bpb'] = ground_stats['balls']/(ground_stats['fours']+ground_stats['sixes'])
ground_stats['dot_%'] = 100*ground_stats['dots']/ground_stats['balls']
ground_stats['extra_per_over'] = 6*ground_stats['extras']/ground_stats['balls']

ground_stats['wkt_pb'] = 1/ground_stats['bowl_strike_rate']

ground_stats['one_pb'] = ground_stats['ones']/ground_stats['balls']
ground_stats['two_pb'] = ground_stats['twos']/ground_stats['balls']
ground_stats['three_pb'] = ground_stats['threes']/ground_stats['balls']
ground_stats['four_pb'] = ground_stats['fours']/ground_stats['balls']
ground_stats['six_pb'] = ground_stats['sixes']/ground_stats['balls']
ground_stats['dot_pb'] = ground_stats['dots']/ground_stats['balls']

ground_stats['wide_pb'] = ground_stats['wides']/ground_stats['balls']
ground_stats['no_pb'] = ground_stats['noballs']/ground_stats['balls']
ground_stats['bye_pb'] = ground_stats['byes']/ground_stats['balls']
ground_stats['legbye_pb'] = ground_stats['legbyes']/ground_stats['balls']

pb_cols = [column for column in ground_stats.columns if '_pb' in str(column)]
pb_cols_div = [column for column in ground_stats.columns if '_pb' in str(column) and column!='wkt_pb']
print(len(pb_cols))

ground_stats

11


,venue,phase,innings,num_innings,runs,balls,wickets,fours,sixes,dots,extras,ones,twos,threes,wides,noballs,byes,legbyes,run_rate,strike_rate,bowl_strike_rate,bpb,dot_%,extra_per_over,wkt_pb,one_pb,two_pb,three_pb,four_pb,six_pb,dot_pb,wide_pb,no_pb,bye_pb,legbye_pb
0,CSK,death,1,17,811,517,39,64,33,153,20,236,43,5,16,1,2,1,9.411992,156.866538,13.256410,5.329897,29.593810,0.232108,0.075435,0.456480,0.083172,0.009671,0.123791,0.063830,0.295938,0.030948,0.001934,0.003868,0.001934
1,CSK,death,2,16,682,417,36,42,41,155,16,137,32,17,7,0,1,8,9.812950,163.549161,11.583333,5.024096,37.170264,0.230216,0.086331,0.328537,0.076739,0.040767,0.100719,0.098321,0.371703,0.016787,0.000000,0.002398,0.019185
2,CSK,middle,1,17,1261,883,37,103,34,218,25,481,56,9,15,3,4,3,8.568516,142.808607,23.864865,6.445255,24.688562,0.169875,0.041903,0.544734,0.063420,0.010193,0.116648,0.038505,0.246886,0.016988,0.003398,0.004530,0.003398
3,CSK,middle,2,17,1272,869,38,71,56,286,41,369,76,30,16,3,12,10,8.782509,146.375144,22.868421,6.842520,32.911392,0.283084,0.043728,0.424626,0.087457,0.034522,0.081703,0.064442,0.329114,0.018412,0.003452,0.013809,0.011507
4,CSK,pp,1,17,876,586,30,110,17,235,35,211,29,10,22,4,1,8,8.969283,149.488055,19.533333,4.614173,40.102389,0.358362,0.051195,0.360068,0.049488,0.017065,0.187713,0.029010,0.401024,0.037543,0.006826,0.001706,0.013652
5,CSK,pp,2,17,850,578,26,83,36,266,39,196,26,5,28,6,2,3,8.823529,147.058824,22.230769,4.857143,46.020761,0.404844,0.044983,0.339100,0.044983,0.008651,0.143599,0.062284,0.460208,0.048443,0.010381,0.003460,0.005190
6,DC,death,1,14,812,462,13,69,40,118,13,210,29,5,9,0,0,4,10.545455,175.757576,35.538462,4.238532,25.541126,0.168831,0.028139,0.454545,0.062771,0.010823,0.149351,0.086580,0.255411,0.019481,0.000000,0.000000,0.008658
7,DC,death,2,14,689,398,25,43,39,124,17,162,31,14,13,2,0,2,10.386935,173.115578,15.920000,4.853659,31.155779,0.256281,0.062814,0.407035,0.077889,0.035176,0.108040,0.097990,0.311558,0.032663,0.005025,0.000000,0.005025
8,DC,middle,1,14,1145,722,26,86,54,217,25,329,45,11,20,0,0,5,9.515235,158.587258,27.769231,5.157143,30.055402,0.207756,0.036011,0.455679,0.062327,0.015235,0.119114,0.074792,0.300554,0.027701,0.000000,0.000000,0.006925
9,DC,middle,2,14,1083,723,33,63,49,218,30,339,51,22,15,4,0,11,8.987552,149.792531,21.909091,6.455357,30.152144,0.248963,0.045643,0.468880,0.070539,0.030429,0.087137,0.067773,0.301521,0.020747,0.005533,0.000000,0.015214


In [11]:
ground_stats['pb_sum'] = ground_stats[pb_cols].apply(lambda x: x.sum(), axis=1)

ground_stats['wkt_prob'] = ground_stats['wkt_pb']/1

for col in pb_cols_div:
    rem_sum = (ground_stats['pb_sum']-ground_stats['wkt_pb'])
    multiplier = (1-ground_stats['wkt_prob'])/rem_sum
    ground_stats[str(col).replace('_pb','_prob')] = ground_stats[col]*multiplier

prob_cols = [column for column in ground_stats.columns if '_prob' in str(column)]

for col in ground_stats.columns[[0,1,2,18,19,20,21,22]]:
    prob_cols.append(col)

ground_stats_trimmed = ground_stats[prob_cols]

In [12]:
ground_stats_trimmed

,wkt_prob,one_prob,two_prob,three_prob,four_prob,six_prob,dot_prob,wide_prob,no_prob,bye_prob,legbye_prob,venue,phase,innings,run_rate,strike_rate,bowl_strike_rate,bpb,dot_%
0,0.075435,0.393858,0.071762,0.008344,0.106809,0.055073,0.255340,0.026702,0.001669,0.003338,0.001669,CSK,death,1,9.411992,156.866538,13.256410,5.329897,29.593810
1,0.086331,0.284483,0.066449,0.035301,0.087214,0.085137,0.321861,0.014536,0.000000,0.002077,0.016612,CSK,death,2,9.812950,163.549161,11.583333,5.024096,37.170264
2,0.041903,0.497673,0.057941,0.009312,0.106570,0.035179,0.225556,0.015520,0.003104,0.004139,0.003104,CSK,middle,1,8.568516,142.808607,23.864865,6.445255,24.688562
3,0.043728,0.379832,0.078231,0.030881,0.073084,0.057644,0.294396,0.016470,0.003088,0.012352,0.010294,CSK,middle,2,8.782509,146.375144,22.868421,6.842520,32.911392
4,0.051195,0.309425,0.042528,0.014665,0.161312,0.024930,0.344620,0.032262,0.005866,0.001466,0.011732,CSK,pp,1,8.969283,149.488055,19.533333,4.614173,40.102389
5,0.044983,0.287532,0.038142,0.007335,0.121761,0.052812,0.390222,0.041076,0.008802,0.002934,0.004401,CSK,pp,2,8.823529,147.058824,22.230769,4.857143,46.020761
6,0.028139,0.421675,0.058231,0.010040,0.138550,0.080319,0.236941,0.018072,0.000000,0.000000,0.008032,DC,death,1,10.545455,175.757576,35.538462,4.238532,25.541126
7,0.062814,0.353079,0.067565,0.030513,0.093719,0.085001,0.270258,0.028334,0.004359,0.000000,0.004359,DC,death,2,10.386935,173.115578,15.920000,4.853659,31.155779
8,0.036011,0.413497,0.056557,0.013825,0.108087,0.067869,0.272732,0.025137,0.000000,0.000000,0.006284,DC,middle,1,9.515235,158.587258,27.769231,5.157143,30.055402
9,0.045643,0.419076,0.063047,0.027197,0.077881,0.060574,0.269495,0.018543,0.004945,0.000000,0.013598,DC,middle,2,8.987552,149.792531,21.909091,6.455357,30.152144


In [13]:
ground_stats_trimmed.to_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/ground_stats.csv',
                           index=None)